In [1]:
#Shared/Utility scripts
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import os
import pathlib
import glob
import boto3

import sys
sys.path.insert(0, '..') #sys allows for the .ipynb file to connect to the shared folder files
from shared_scripts import AWS_transfer

HOME = os.path.expanduser('~')
KEYPATH = "NWM_ML/AWSaccessKeys.csv"
ACCESS = pd.read_csv(f"{HOME}/{KEYPATH}")

#start session
SESSION = boto3.Session(
    aws_access_key_id=ACCESS['Access key ID'][0],
    aws_secret_access_key=ACCESS['Secret access key'][0],
)
S3 = SESSION.resource('s3')
#AWS BUCKET information
BUCKET_NAME = 'streamflow-app-data'
BUCKET = S3.Bucket(BUCKET_NAME)


In [4]:
BUCKET

s3.Bucket(name='streamflow-app-data')

In [24]:
#Need to load all csvs from a folder, convert to parquet, send to appropriate folder in AWS
path = f"{HOME}/NWM_ML/Data/Processed"
files = list(pathlib.Path(path).glob('*.pkl'))
files

[PosixPath('/home/rjohnson18/NWM_ML/Data/Processed/Precip_Total.pkl'),
 PosixPath('/home/rjohnson18/NWM_ML/Data/Processed/Temperature.pkl'),
 PosixPath('/home/rjohnson18/NWM_ML/Data/Processed/flow.pkl')]

In [23]:
#for excell files
for file in files:
    newfile = f"{file.stem}.parquet"
    newfilepath = f"{path}/{newfile}"
    print(newfilepath)
    try:
        f = pd.read_csv(file)
    except:
        f = pd.read_excel(file)
    table = pa.Table.from_pandas(f)
    pq.write_table(table,newfilepath, compression='BROTLI')

    #send to aws
    AWSpath = f"NWM_ML{path.split('NWM_ML')[-1].split('.')[0]}"
    S3.meta.client.upload_file(Filename= newfilepath, Bucket=BUCKET_NAME, Key=f"{AWSpath}/{newfile}")
    

/home/rjohnson18/NWM_ML/Data/Processed/Climate.parquet
/home/rjohnson18/NWM_ML/Data/Processed/NWM_ML_Pred_DF1.parquet
/home/rjohnson18/NWM_ML/Data/Processed/NWM_ML_Training_DF.parquet
/home/rjohnson18/NWM_ML/Data/Processed/raw_training_data.parquet
/home/rjohnson18/NWM_ML/Data/Processed/Pred_DF1.parquet
/home/rjohnson18/NWM_ML/Data/Processed/StreamStats_All.parquet
/home/rjohnson18/NWM_ML/Data/Processed/Training_DF1.parquet
/home/rjohnson18/NWM_ML/Data/Processed/Training_DF2.parquet


In [28]:
#send to aws - non parquet
for file in files:
    newfile = f"{file.stem}.pkl"
    AWSpath = f"NWM_ML{path.split('NWM_ML')[-1].split('.')[0]}"
    S3.meta.client.upload_file(Filename= file, Bucket=BUCKET_NAME, Key=f"{AWSpath}/{newfile}")

In [21]:
newfile

'stream_stats.parquet'